In [2]:
import cv2
import mediapipe as mp
import os
import random
import numpy as np

# 初始化 MediaPipe Pose 模組
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()
mp_drawing = mp.solutions.drawing_utils

# 資料夾路徑
image_folder = r"C:\Users\0423d\pythonWork\1131D2Final\orgimg"

# 函數：從圖片提取姿勢關鍵點
def extract_pose_from_image(image_path):
    image = cv2.imread(image_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    result = pose.process(image_rgb)
    if result.pose_landmarks:
        return [(lm.x, lm.y, lm.z) for lm in result.pose_landmarks.landmark]
    return None

# 讀取資料夾中的所有圖片並提取姿勢
pose_data = []
image_paths = [os.path.join(image_folder, f) for f in os.listdir(image_folder) if f.endswith(('.png', '.jpg', '.jpeg'))]
for img_path in image_paths:
    pose_points = extract_pose_from_image(img_path)
    if pose_points:
        pose_data.append(pose_points)

if not pose_data:
    print("沒有成功提取任何姿勢數據！請檢查圖片或MediaPipe設置。")
    exit()

# 隨機選擇一個姿勢
target_pose = random.choice(pose_data)

# 函數：在影像上繪製姿勢
def draw_pose_on_frame(frame, pose_points):
    height, width, _ = frame.shape
    for point in pose_points:
        x, y, z = point
        x = int(x * width)
        y = int(y * height)
        cv2.circle(frame, (x, y), 5, (0, 255, 0), -1)

# 開啟 Webcam，顯示即時影像與目標姿勢
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("無法讀取Webcam影像。")
        break

    # 在影像上疊加目標姿勢
    draw_pose_on_frame(frame, target_pose)

    # 顯示影像
    cv2.imshow("Pose Matching", frame)

    # 按下 'q' 鍵退出
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# 釋放資源
cap.release()
cv2.destroyAllWindows()


In [ ]:
import cv2
print(cv2.__version__)


In [5]:
# Cam辨識肢體
# ========
import cv2
import mediapipe as mp

# 初始化 MediaPipe Pose 模組
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils

# 啟動 Webcam
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("無法接收到影像，請檢查相機設置。")
        break

    # 將影像轉換為 RGB
    image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # 偵測人體姿勢
    results = pose.process(image_rgb)

    # 在影像中繪製姿勢骨架
    if results.pose_landmarks:
        mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

    # 顯示影像
    cv2.imshow('Pose Estimation', frame)

    # 按下 'q' 鍵停止
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# 釋放攝像頭並關閉視窗
cap.release()
cv2.destroyAllWindows()


In [ ]:
# 可顯示01
# ========
import cv2
import mediapipe as mp
import os
import random

# 初始化 MediaPipe Pose 模組
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=True, min_detection_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils

# 資料夾路徑
image_folder = r"C:\Users\0423d\pythonWork\1131D2Final\orgimg"

# 函數：從圖片中提取姿勢關鍵點
def extract_pose_from_image(image_path):
    image = cv2.imread(image_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    result = pose.process(image_rgb)
    if result.pose_landmarks:
        return [(lm.x, lm.y, lm.z) for lm in result.pose_landmarks.landmark]
    return None

# 讀取資料夾中的所有圖片並提取姿勢
image_paths = [os.path.join(image_folder, f) for f in os.listdir(image_folder) if f.endswith(('.png', '.jpg', '.jpeg'))]

if not image_paths:
    print("資料夾中沒有符合圖片格式的檔案。")
    exit()

# 隨機選擇一張圖片
selected_image_path = random.choice(image_paths)

# 將選擇的圖片的姿勢關鍵點提取出來
pose_points = extract_pose_from_image(selected_image_path)
if not pose_points:
    print(f"圖片 {selected_image_path} 中沒有偵測到姿勢！")
    exit()

# 頭部、手部、腳部的關鍵點索引
head_index = 0
left_shoulder_index = 11
right_shoulder_index = 12
left_elbow_index = 13
right_elbow_index = 14
left_wrist_index = 15
right_wrist_index = 16
left_hip_index = 23
right_hip_index = 24
left_knee_index = 25
right_knee_index = 26
left_ankle_index = 27
right_ankle_index = 28

# 在畫面中繪製姿勢的火柴人骨架
def draw_stick_figure(frame, landmarks):
    # 確保 landmarks 不是空的
    if landmarks is None or len(landmarks) < 29:
        return

    # 連接骨架的線
    connections = [
        (11, 13), (13, 15), (12, 14), (14, 16),  # 上肢：肩膀到手腕
        (23, 25), (25, 27), (24, 26), (26, 28),  # 下肢：髖部到腳踝
        (11, 12), (23, 24),  # 肩膀與髖部連接
        (11, 24), (12, 23),  # 肩膀與髖部之間的對角線
    ]

    # 繪製每一條骨架線
    for connection in connections:
        start = landmarks[connection[0]]
        end = landmarks[connection[1]]
        start_point = (int(start[0] * frame.shape[1]), int(start[1] * frame.shape[0]))  # start[0] 是 x，start[1] 是 y
        end_point = (int(end[0] * frame.shape[1]), int(end[1] * frame.shape[0]))  # end[0] 是 x，end[1] 是 y
        cv2.line(frame, start_point, end_point, (0, 255, 0), 2)  # 用綠色畫線

    # 繪製關鍵點
    for idx, point in enumerate(landmarks):
        x = int(point[0] * frame.shape[1])
        y = int(point[1] * frame.shape[0])
        cv2.circle(frame, (x, y), 5, (0, 0, 255), -1)  # 用紅色畫關鍵點

        # 標註頭、肩膀、手、腳
        if idx == head_index:
            cv2.putText(frame, 'Head', (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
        elif idx == left_shoulder_index or idx == right_shoulder_index:
            cv2.putText(frame, 'Shoulder', (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
        elif idx == left_wrist_index or idx == right_wrist_index:
            cv2.putText(frame, 'Wrist', (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
        elif idx == left_ankle_index or idx == right_ankle_index:
            cv2.putText(frame, 'Ankle', (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)

# 開啟 Webcam
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("無法讀取Webcam影像。")
        break

    # 在 Webcam 視窗中繪製已選擇圖片的火柴人骨架
    draw_stick_figure(frame, pose_points)

    # 顯示影像
    cv2.imshow("Pose Estimation - Stick Figure", frame)

    # 按下 'q' 鍵退出
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# 釋放資源
cap.release()
cv2.destroyAllWindows()


In [5]:
# 可顯示，但不能關原圖
# ====
import cv2
import mediapipe as mp
import os
import random

# 初始化 MediaPipe Pose 模組
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=True, min_detection_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils

# 資料夾路徑
image_folder = r"C:\Users\0423d\pythonWork\1131D2Final\orgimg"

# 函數：從圖片中提取姿勢關鍵點
def extract_pose_from_image(image_path):
    image = cv2.imread(image_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    result = pose.process(image_rgb)
    if result.pose_landmarks:
        return [(lm.x, lm.y, lm.z) for lm in result.pose_landmarks.landmark]
    return None

# 讀取資料夾中的所有圖片並提取姿勢
image_paths = [os.path.join(image_folder, f) for f in os.listdir(image_folder) if f.endswith(('.png', '.jpg', '.jpeg'))]

if not image_paths:
    print("資料夾中沒有符合圖片格式的檔案。")
    exit()

# 隨機選擇一張圖片
selected_image_path = random.choice(image_paths)

# 將選擇的圖片的姿勢關鍵點提取出來
pose_points = extract_pose_from_image(selected_image_path)
if not pose_points:
    print(f"圖片 {selected_image_path} 中沒有偵測到姿勢！")
    exit()

# 在畫面中繪製姿勢的火柴人骨架
def draw_stick_figure(frame, landmarks, scale_factor=1):
    # 確保 landmarks 不是空的
    if landmarks is None or len(landmarks) < 29:
        return

    # 連接骨架的線
    connections = [
        (11, 13), (13, 15), (12, 14), (14, 16),  # 上肢：肩膀到手腕
        (23, 25), (25, 27), (24, 26), (26, 28),  # 下肢：髖部到腳踝
        (11, 12), (23, 24),  # 肩膀與髖部連接
        (11, 24), (12, 23),  # 肩膀與髖部之間的對角線
    ]

    # 繪製每一條骨架線
    for connection in connections:
        start = landmarks[connection[0]]
        end = landmarks[connection[1]]
        start_point = (int(start[0] * frame.shape[1] * scale_factor), int(start[1] * frame.shape[0] * scale_factor))
        end_point = (int(end[0] * frame.shape[1] * scale_factor), int(end[1] * frame.shape[0] * scale_factor))
        cv2.line(frame, start_point, end_point, (0, 255, 0), 2)  # 用綠色畫線

    # 繪製關鍵點
    for idx, point in enumerate(landmarks):
        x = int(point[0] * frame.shape[1] * scale_factor)
        y = int(point[1] * frame.shape[0] * scale_factor)
        cv2.circle(frame, (x, y), 5, (0, 0, 255), -1)  # 用紅色畫關鍵點

# 開啟 Webcam
cap = cv2.VideoCapture(0)

# 讀取原圖
original_image = cv2.imread(selected_image_path)

# 左右翻轉原圖
original_image = cv2.flip(original_image, 1)

# 計算適應螢幕的圖片尺寸
height, width = original_image.shape[:2]
max_dim = 600  # 最大寬度或高度
if max(height, width) > max_dim:
    scale_factor = max_dim / max(height, width)
    new_width = int(width * scale_factor)
    new_height = int(height * scale_factor)
    original_image = cv2.resize(original_image, (new_width, new_height))

# 開啟新視窗顯示原圖，並等待顯示
cv2.imshow("Original Image", original_image)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("無法讀取Webcam影像。")
        break

    # 在 Webcam 視窗中繪製已選擇圖片的火柴人骨架
    scale_factor = 1  # 不縮放火柴人骨架，與圖片範圍一致
    draw_stick_figure(frame, pose_points, scale_factor)

    # 顯示影像
    cv2.imshow("Pose Estimation - Stick Figure", frame)

    # 如果按下 'l' 鍵，顯示原圖
    if cv2.waitKey(1) & 0xFF == ord('l'):
        # 顯示左右翻轉後的原圖
        cv2.imshow("Original Image", original_image)

    # 按下 'q' 鍵退出
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# 釋放資源
cap.release()
cv2.destroyAllWindows()


In [6]:
import cv2
import mediapipe as mp
import os
import random

# 初始化 MediaPipe Pose 模組
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=True, min_detection_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils

# 資料夾路徑
image_folder = r"C:\Users\0423d\pythonWork\1131D2Final\orgimg"

# 函數：從圖片中提取姿勢關鍵點
def extract_pose_from_image(image_path):
    image = cv2.imread(image_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    result = pose.process(image_rgb)
    if result.pose_landmarks:
        return [(lm.x, lm.y, lm.z) for lm in result.pose_landmarks.landmark]
    return None

# 讀取資料夾中的所有圖片並提取姿勢
image_paths = [os.path.join(image_folder, f) for f in os.listdir(image_folder) if f.endswith(('.png', '.jpg', '.jpeg'))]

if not image_paths:
    print("資料夾中沒有符合圖片格式的檔案。")
    exit()

# 隨機選擇一張圖片
selected_image_path = random.choice(image_paths)

# 將選擇的圖片的姿勢關鍵點提取出來
pose_points = extract_pose_from_image(selected_image_path)
if not pose_points:
    print(f"圖片 {selected_image_path} 中沒有偵測到姿勢！")
    exit()

# 在畫面中繪製姿勢的火柴人骨架
def draw_stick_figure(frame, landmarks, scale_factor=1):
    if landmarks is None or len(landmarks) < 29:
        return

    connections = [
        (11, 13), (13, 15), (12, 14), (14, 16),
        (23, 25), (25, 27), (24, 26), (26, 28),
        (11, 12), (23, 24), (11, 24), (12, 23),
    ]

    for connection in connections:
        start = landmarks[connection[0]]
        end = landmarks[connection[1]]
        start_point = (int(start[0] * frame.shape[1] * scale_factor), int(start[1] * frame.shape[0] * scale_factor))
        end_point = (int(end[0] * frame.shape[1] * scale_factor), int(end[1] * frame.shape[0] * scale_factor))
        cv2.line(frame, start_point, end_point, (0, 255, 0), 2)

    for point in landmarks:
        x = int(point[0] * frame.shape[1] * scale_factor)
        y = int(point[1] * frame.shape[0] * scale_factor)
        cv2.circle(frame, (x, y), 5, (0, 0, 255), -1)

# 開啟 Webcam
cap = cv2.VideoCapture(0)

# 讀取原圖並進行翻轉
original_image = cv2.imread(selected_image_path)
original_image = cv2.flip(original_image, 1)

# 調整原圖尺寸
height, width = original_image.shape[:2]
max_dim = 600
if max(height, width) > max_dim:
    scale_factor = max_dim / max(height, width)
    new_width = int(width * scale_factor)
    new_height = int(height * scale_factor)
    original_image = cv2.resize(original_image, (new_width, new_height))

# 狀態變數：是否顯示原圖
show_original_image = False

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("無法讀取Webcam影像。")
        break

    # 繪製火柴人骨架在 Webcam 畫面上
    draw_stick_figure(frame, pose_points)

    # 顯示 Webcam 畫面
    cv2.imshow("Pose Estimation - Stick Figure", frame)

    # 按下 'l' 鍵切換原圖顯示狀態
    key = cv2.waitKey(1) & 0xFF
    if key == ord('l'):
        show_original_image = not show_original_image
        if show_original_image:
            cv2.imshow("Original Image", original_image)
        else:
            cv2.destroyWindow("Original Image")

    # 按下 'q' 鍵退出
    if key == ord('q'):
        break

# 釋放資源
cap.release()
cv2.destroyAllWindows()


In [1]:
import cv2
import mediapipe as mp
import os
import random

# 初始化 MediaPipe Pose 模組
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=True, min_detection_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils

# 資料夾路徑
image_folder = r"C:\Users\0423d\pythonWork\1131D2Final\orgimg"

# 函數：從圖片中提取姿勢關鍵點
def extract_pose_from_image(image_path):
    image = cv2.imread(image_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    result = pose.process(image_rgb)
    if result.pose_landmarks:
        return [(lm.x, lm.y, lm.z) for lm in result.pose_landmarks.landmark]
    return None

# 函數：載入圖片並進行必要處理
def load_new_image():
    selected_image_path = random.choice(image_paths)
    pose_points = extract_pose_from_image(selected_image_path)
    if not pose_points:
        print(f"圖片 {selected_image_path} 中沒有偵測到姿勢！")
        return None, None

    # 載入原圖並翻轉
    original_image = cv2.imread(selected_image_path)
    original_image = cv2.flip(original_image, 1)

    # 調整原圖尺寸
    height, width = original_image.shape[:2]
    max_dim = 600
    if max(height, width) > max_dim:
        scale_factor = max_dim / max(height, width)
        new_width = int(width * scale_factor)
        new_height = int(height * scale_factor)
        original_image = cv2.resize(original_image, (new_width, new_height))

    return pose_points, original_image

# 讀取資料夾中的所有圖片路徑
image_paths = [os.path.join(image_folder, f) for f in os.listdir(image_folder) if f.endswith(('.png', '.jpg', '.jpeg'))]
if not image_paths:
    print("資料夾中沒有符合圖片格式的檔案。")
    exit()

# 初始載入一張圖片和姿勢
pose_points, original_image = load_new_image()
if pose_points is None or original_image is None:
    print("初始圖片載入失敗。")
    exit()

# 在畫面中繪製姿勢的火柴人骨架
def draw_stick_figure(frame, landmarks, scale_factor=1):
    if landmarks is None or len(landmarks) < 29:
        return

    connections = [
        (11, 13), (13, 15), (12, 14), (14, 16),
        (23, 25), (25, 27), (24, 26), (26, 28),
        (11, 12), (23, 24), (11, 23), (12, 24),
    ]

    for connection in connections:
        start = landmarks[connection[0]]
        end = landmarks[connection[1]]
        start_point = (int(start[0] * frame.shape[1] * scale_factor), int(start[1] * frame.shape[0] * scale_factor))
        end_point = (int(end[0] * frame.shape[1] * scale_factor), int(end[1] * frame.shape[0] * scale_factor))
        cv2.line(frame, start_point, end_point, (0, 255, 0), 2)

    for point in landmarks:
        x = int(point[0] * frame.shape[1] * scale_factor)
        y = int(point[1] * frame.shape[0] * scale_factor)
        cv2.circle(frame, (x, y), 5, (0, 0, 255), -1)

# 開啟 Webcam
cap = cv2.VideoCapture(0)

# 狀態變數：是否顯示原圖
show_original_image = False

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("無法讀取Webcam影像。")
        break

    # 繪製火柴人骨架在 Webcam 畫面上
    draw_stick_figure(frame, pose_points)

    # 顯示 Webcam 畫面
    cv2.imshow("Pose Estimation - Stick Figure", frame)

    # 按下 'l' 鍵切換原圖顯示狀態
    key = cv2.waitKey(1) & 0xFF
    if key == ord('l'):
        show_original_image = not show_original_image
        if show_original_image:
            cv2.imshow("Original Image", original_image)
        else:
            cv2.destroyWindow("Original Image")

    # 按下 'r' 鍵重新選擇圖片並更新火柴人
    if key == ord('r'):
        new_pose_points, new_original_image = load_new_image()
        if new_pose_points and new_original_image is not None:
            pose_points = new_pose_points
            original_image = new_original_image
            if show_original_image:
                cv2.imshow("Original Image", original_image)

    # 按下 'q' 鍵退出
    if key == ord('q'):
        break

# 釋放資源
cap.release()
cv2.destroyAllWindows()
